In [1]:
# downloads financial data (both current snapshot and time-series) via API using Yahoo Finance

import pandas as pd
import yfinance as yf
import datetime as datetime
import plotly.express as px

In [2]:
# downloads current snapshot (i.e. non-time series) via API

tickers = ['^GSPC', 'IEUR', 'MSFT', 'XOM'] # list tickers that you wanna download the data for
#tickers = ['^GSPC', 'XWD.TO', 'IEUR', 'EWU', '^HSI', 'VXUS', 'CL=F', 'GC=F']
fields_list = ['ticker', 'shortName', 'sector', 'industry', 'country', 'logo_url', 'floatShares', 'currency', 'financialCurrency', 'fullTimeEmployees', 'currentPrice', 'bid', 'ask', 'fiftyTwoWeekHigh', 'fiftyTwoWeekLow', 'totalRevenue', 'ebitda', 'netIncomeToCommon', 'totalDebt', 'totalCash', 'trailingEps', 'marketCap', 'enterpriseValue', 'enterpriseToEbitda', 'trailingPE', 'pegRatio', 'trailingPegRatio', 'dividendYield', 'fiveYearAvgDividendYield', 'payoutRatio', 'earningsGrowth', 'totalCashPerShare', 'ebitdaMargins', 'profitMargins', 'debtToEquity', 'recommendationMean', 'targetHighPrice', 'targetLowPrice', 'targetMedianPrice', 'targetMedianPrice', 'forwardPE', 'forwardEps', 'shortPercentOfFloat'] # creates a list of field names that I want to download the data for
dict_curr_data ={} # createsa an empty dictionary to host tickers as keys and API data values as DataFrames
df_curr_data = [] # deletes DataFrame in case it was created earlier

for ticker in tickers:
    ticker_obj = yf.Ticker(ticker) # downloads current data via API
    # converts info() output from dictionary to DataFrame
    temp_curr_data = pd.DataFrame.from_dict(ticker_obj.info, orient='index')
    temp_curr_data.reset_index(inplace=True)
    temp_curr_data.columns = ["Field", "Value"]
    dict_curr_data[ticker] = temp_curr_data # adds (ticker, dataframe) to the main dictionary

# combine this dictionary of dataframes into a single dataframe:
df_curr_data = pd.concat(dict_curr_data)
df_curr_data.reset_index(inplace=True)
df_curr_data.drop(['level_1'], axis=1, inplace=True) # deletes 'level_1' column
df_curr_data = df_curr_data[df_curr_data['Field'].isin(fields_list)] # deletes all rows where values under 'Field' are not listed in 'fields_list'
# TODO for each ticker code add rows to calculate ND/Ebitda and Ebitda/employee
# myNetDebtToEbitda = (totalDebt - totalCash) / ebitda
# myEbitdaPerEmployee = ebitda / fullTimeEmployees
df_curr_data

,level_0,Field,Value
1,^GSPC,shortName,S&P 500
14,^GSPC,payoutRatio,None
36,^GSPC,currency,USD
41,^GSPC,marketCap,None
47,^GSPC,ask,4121.58
...,...,...,...
480,XOM,fiftyTwoWeekLow,57.96
481,XOM,bid,109.6
483,XOM,dividendYield,0.0329
489,XOM,logo_url,https://logo.clearbit.com/corporate.exxonmobil...


In [3]:
# charts pricing data by importing OHLC data and saving it as 'df_px_close' DataFrame
df_px_close = [] # deletes DataFrame in case it was created earlier
df_px_close = pd.DataFrame() # creates empty DataFrame to store closing stock price data
for ticker in tickers: # loops through each ticker to download its closing share price and add it under a newe column of the DataFrame
    df_px_close[ticker] = yf.Ticker(ticker).history(period='5y').Close
    # another way to do it is here: https://www.allthesnippets.com/notes/finance/calculating_cumulative_returns_of_stocks_with_python_and_pandas.html

# charts prices of all tickers on a single graph
fig_px_close = px.line(df_px_close, 
                    x=df_px_close.index, 
                    y=df_px_close.columns,
                    height=450, 
                    width=750,
                    template='plotly_dark'
) # builds the chart
fig_px_close.show() # displays the chart

In [39]:
# charts cumulative return of all tickers in a single chart. Full instructions here: https://www.allthesnippets.com/notes/finance/calculating_cumulative_returns_of_stocks_with_python_and_pandas.html

# Calculates daily & cumulative returns
df_daily_returns = df_px_close.pct_change() # computes daily returns for each security
df_daily_returns = df_daily_returns[1:] # deletes first row with NaN values
df_cum_daily_returns = (1 + df_daily_returns).cumprod() - 1 # calculates cumulative daily returns
df_cum_daily_returns = df_cum_daily_returns.reset_index() # to add row_index to the DataFrame

# Transforms cumulative returns data for charting
df_px_cum_return = df_cum_daily_returns.melt(id_vars=['Date'], value_vars=tickers, var_name='Ticker', value_name='Cumulative Return')
df_px_cum_return['Cumulative Return Percent'] = df_px_cum_return['Cumulative Return'] * 100

# Charts cumulative returns
fig_px_cum_return = px.line(df_px_cum_return, x='Date',
              y='Cumulative Return Percent', color='Ticker',
              title='Performance: Daily Cumulative Returns',
              labels={'Cumulative Return Percent':'daily cumulative returns (%)', },
              height=450, 
              width=750,
              template='plotly_dark'
) # builds the chart
fig_px_cum_return.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
) # removes gridlines from the chart
fig_px_cum_return.show() # displays the chart

In [99]:
# calculates (cumulative?) return per security using formula: (last price - initial price) / initial price
cum_return = round(((df_px_close.iloc[-1] - df_px_close.iloc[0]) / df_px_close.iloc[0]) * 100)
cum_return

^GSPC     54.0
CL=F      42.0
GC=F      43.0
VXUS      11.0
XWD.TO    51.0
IEUR      15.0
EWU       11.0
dtype: float64

In [ ]:
# # simple syntax to download current (non-time series) data using Yahoo Finance API

# tickers = 'MSFT'
# ticker_obj = yf.Ticker(tickers)
# stockinfo = ticker_obj.info

# # to view all entries in stockinfo dictionary as one line per item
# for key, value in stockinfo.items():
#     print(key, ":", value)

In [ ]:
# to import current data
shortName = stockinfo['shortName']
sector = stockinfo['sector']
industry = stockinfo['industry']
country = stockinfo['country']
logo_url = stockinfo['logo_url']
floatShares = stockinfo['floatShares']
currency = stockinfo['currency']
financialCurrency = stockinfo['financialCurrency']
fullTimeEmployees = stockinfo['fullTimeEmployees']
currentPrice = stockinfo['currentPrice']
bid = stockinfo['bid']
ask = stockinfo['ask']
fiftyTwoWeekHigh = stockinfo['fiftyTwoWeekHigh']
fiftyTwoWeekLow = stockinfo['fiftyTwoWeekLow']
totalRevenue = stockinfo['totalRevenue']
ebitda = stockinfo['ebitda']
netIncomeToCommon = stockinfo['netIncomeToCommon']
totalDebt = stockinfo['totalDebt']
totalCash = stockinfo['totalCash']
trailingEps = stockinfo['trailingEps']
marketCap = stockinfo['marketCap']
enterpriseValue = stockinfo['enterpriseValue']
enterpriseToEbitda = stockinfo['enterpriseToEbitda']
trailingPE = stockinfo['trailingPE']
pegRatio = stockinfo['pegRatio']
trailingPegRatio = stockinfo['trailingPegRatio']
dividendYield = stockinfo['dividendYield']
fiveYearAvgDividendYield = stockinfo['fiveYearAvgDividendYield']
payoutRatio = stockinfo['payoutRatio']
earningsGrowth = stockinfo['earningsGrowth']
totalCashPerShare = stockinfo['totalCashPerShare']
ebitdaMargins = stockinfo['ebitdaMargins']
profitMargins = stockinfo['profitMargins'] 
debtToEquity = stockinfo['debtToEquity']
recommendationMean = stockinfo['recommendationMean']
targetHighPrice = stockinfo['targetHighPrice']
targetLowPrice = stockinfo['targetLowPrice']
targetMedianPrice = stockinfo['targetMedianPrice']
targetMedianPrice = stockinfo['targetMedianPrice']
forwardPE = stockinfo['forwardPE']
forwardEps = stockinfo['forwardEps']
shortPercentOfFloat = stockinfo['shortPercentOfFloat']
myNetDebtToEbitda = (totalDebt - totalCash) / ebitda
myEbitdaPerEmployee = ebitda / fullTimeEmployees


print(shortName)

Microsoft Corporation
